In [7]:
import sys
sys.path.insert(1, '/home/lruffati/Documents/0.University/0.SistemiElettorali/')
import src
from src import Commons

from src.Metaclasses import *
import src.GlobalVars as GlobalVars
import random
import pandas as pd

In [8]:
class ExampleNode:
    def __init__(self):
        print("Created example: ", self)
        
    def propose(self, *a, **k):
        return pd.DataFrame({'ac':['aa','ba'],'s':[1,2]}), {'polent1':{'gen_info': True}}

In [9]:
class ActualHub:
    def get_subdivisions(self, name_subs, *args, **k):
        print("Get subs: ", self, name_subs, args, k)
        return ["c1", "c2"]
    def get_instance(self, name_class, name_inst):
        print("Get inst: ", self, name_class, name_inst)
        return ExampleSubLvl(name_inst)

class ExampleSelf:
    def propose(self, *args, **kwargs):
        print("Propose:", self, args, kwargs)
        return pd.DataFrame({'parts':['pa','pb','pc'],
                            'seats': [5, 2, 3]}), {'pa':{'votiTot':50},
                                                  'pb':{'votiTot':20},
                                                  'pc':{'votiTot':30}}

class ExampleSubLvl:
    def __init__(self, i):
        print("Create sublv: ", i, self)
        self.id = i
    def propose(self, *arg, **kw):
        print("Propose:", self.id, arg, kw)
        if self.id == "c1":
            return pd.DataFrame({'parts':['pa','pc'], 'seats':[3,1]}), {'pa':{'res':0},
                                                                       'pb':{'res':5},
                                                                       'pc':{'res':6}}
        
        if self.id == "c2":
            return pd.DataFrame({'parts':['pa','pb','pc'], 
                                 'seats':[2,1,1]}), {'pa':{'res':0},
                                                                       'pb':{'res':5},
                                                                       'pc':{'res':4}}

In [10]:
def corr_eu(district, ideal, local_distr, local_infos, distr_infos, *gen_infos, **kw):
    tot_from_loc = pd.concat(local_distr.values(), ignore_index=True)
    tot_from_loc.columns = ["party", "seats"]
    print(tot_from_loc)
    tot_from_loc = tot_from_loc.groupby('party').agg('sum')
    print(tot_from_loc)
    from_subs_dict = tot_from_loc.to_dict('index')

    ideal.columns = ["party", "seats"]
    ideal_dict = ideal.set_index('party').to_dict('index')
    
    adjustments = {}
    for k, d in ideal_dict.items():
        adjustments[k] = d['seats'] - from_subs_dict[k]['seats']
    
    remainders_subs = {k:{} for k in adjustments.keys()}
    for sub, sub_inf in local_infos.items():
        for k in adjustments.keys():
            remainders_subs[k][sub] = sub_inf.get(k, {}).get('res', 0)
    
    seats_to_add = [] # circo:[partiti] 
    for k in adjustments.keys():
        q = adjustments[k]
        lis = remainders_subs[k].items() # [(sub, res)]
        for c, _ in sorted(lis, key=lambda x: x[1], reverse=True)[:q]:
            seats_to_add.append((c, k))
    
    ret = {c: df.rename(columns=dict(zip(df.columns, ['party','seggi']))).set_index('party') \
           for c, df in local_distr.items()}
    for c, party in seats_to_add:
        if party not in ret[c].index:
            ret[c].loc[party, 'seggi'] = 0
        ret[c].loc[party, 'seggi'] += 1
    print(ret)
    return {c: df.reset_index() for c, df in ret.items()}, {}, {}

In [11]:
src.Commons.correct_eu = corr_eu

src.GlobalVars.Hub = ActualHub()

# Test operation
Scenario:  
Europee, sono in Nazione, voglio ricavare la distribuzione di lista, sotto ho circoscrizioni

### Propose:
1. In self, mi restituisce 

In [12]:
conf = [{'collect_type': 'liste',
 'corrector': 'Commons.correct_eu',
 'ideal_distribution': 'prop','forward_distribution':True}]

f = src.Metaclasses.lanes.parse_ops_lane("sub_levl", *conf)

f({'self':ExampleSelf()},
  {}, # loc_infos
  {}, # distr_infos
  {}, # other_infos
  distribution=pd.DataFrame())

Info generic:  {}
Info specific:  {}
-------------
Get subs:  <__main__.ActualHub object at 0x7faf60859790> <__main__.ExampleSelf object at 0x7faf60a6c310> ('sub_levl',) {}
Propose: <__main__.ExampleSelf object at 0x7faf60a6c310> ('prop', {}, {}) {'distribution': Empty DataFrame
Columns: []
Index: []}
line 147, subs=  ['c1', 'c2']
Get inst:  <__main__.ActualHub object at 0x7faf60859790> sub_levl c1
Create sublv:  c1 <__main__.ExampleSubLvl object at 0x7faf60855d60>
Propose: c1 ('liste', {}, {'pa': {'votiTot': 50}, 'pb': {'votiTot': 20}, 'pc': {'votiTot': 30}}, {}, {}) {'distribution':   parts  seats
0    pa      5
1    pb      2
2    pc      3}
Get inst:  <__main__.ActualHub object at 0x7faf60859790> sub_levl c2
Create sublv:  c2 <__main__.ExampleSubLvl object at 0x7faf60b21ac0>
Propose: c2 ('liste', {}, {'pa': {'votiTot': 50}, 'pb': {'votiTot': 20}, 'pc': {'votiTot': 30}}, {}, {}) {'distribution':   parts  seats
0    pa      5
1    pb      2
2    pc      3}
  party  seats
0    pa     

(Empty DataFrame
 Columns: []
 Index: [],
 {'c1': {'pa': {'res': 0}, 'pb': {'res': 5}, 'pc': {'res': 6}},
  'c2': {'pa': {'res': 0}, 'pb': {'res': 5}, 'pc': {'res': 4}}},
 {'pa': {'votiTot': 50}, 'pb': {'votiTot': 20}, 'pc': {'votiTot': 30}})

# Propose testing

In [ ]:
def correct(*args, **kwargs):
    print("Correct: ", args, kwargs)

src.GlobalVars.correct = correct

In [ ]:
locals()['correct']

In [ ]:
conf = {'lane': {'plurinominale': {'first_input': 'kwarg',
                            'info_name': 'Circoscrizione',
                            'node_type': 'node',
                            'operations': [{'collect_type': 'lista',
                                            'corrector': 'src.GlobalVars.correct',
                                            'ideal_distribution': '$'}],
                            'sub_level': 'Plurinominale'}}}

f = lanes.parse_lane_fun('plurinominale', **conf['lane']['plurinominale'])

In [ ]:
import pandas as pd
distr = pd.DataFrame({'a':['a','b','c'], 's':[1,2,3]})

In [ ]:
f(Example(), 'plurinominale', {'info':{'much':1,'info':2}}, {}, {}, distribution=distr )